## Web scraping notebook: Discussion stats

In this notebook we retrieve statistics for each bookID:
    - number of quotes
    - number of discussions
    - number of questions

### Part 1: Scrape Data

In [98]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [99]:
BASE_PATH = "../../data/raw/scraped"
filename = f"{BASE_PATH}/discussion/books_discussion_1_27.csv"
iteration = 1

In [100]:
# Load the latest csv
df = pd.read_csv(filename, sep=',', on_bad_lines="warn")

In [101]:
df.isnull().sum()

Unnamed: 0.2          0
Unnamed: 0.1          0
Unnamed: 0            0
bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
num_pages             0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
quotes                1
discussions           1
questions             1
dtype: int64

### !! Warning run this only the first time you launch the scrapping

In [1]:
##if iteration == 1:
#    df['quotes'] = np.nan
#    df['discussions'] = np.nan
#    df['questions'] = np.nan

In [102]:
def missing_stats_indexes(df):
    """ This function returns a list of indexes of rows where any of 'quotes', 'discussions' or 'questions' is missing in df.
    """
    missing_quotes = df[df['quotes'].isnull()].index
    missing_discussions = df[df['discussions'].isnull()].index
    missing_questions = df[df['questions'].isnull()].index
    missing_total_intersection = set(missing_quotes) & set(missing_discussions) & set(missing_questions)
    return list(missing_total_intersection)

def get_discussion_stats(ID, row_number):
    """This function returns quotes discussions and questions value scraped from goodreads for each bookID
    :param ID: bookID to request on goodreads
    :param row_number: row number in dataframe for console logging
    """
    # Request url and get discussion stats
    url = 'https://www.goodreads.com/book/show/' + str(ID)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    discussion_stats = soup.findAll('div', class_='DiscussionCard__stats')

    # Print logs
    missing = missing_stats_indexes(df)
    percentage = (len(missing)/ 11123) * 100
    formatted_percentage = "{:.2f}%".format(percentage)
    print(f"{len(discussion_stats)} -- found on row : {row_number}  {formatted_percentage} missing")

    quotes = discussions = questions = None

    if discussion_stats:
        quotes = discussion_stats[0].text
        if len(discussion_stats) > 1:
            discussions = discussion_stats[1].text
            if len(discussion_stats) > 2:
                questions = discussion_stats[2].text
    print(quotes, ' ', discussions, '  ',questions)
    return quotes, discussions, questions

In [103]:
# Get missing indexes to search
missing = missing_stats_indexes(df)
print("missed ",len(missing))

# Save the actual version of 'df'
filename = f"{BASE_PATH}/discussion/books_discussion_{len(missing)}_{iteration}.csv"
df.to_csv(filename)

# Start a new iteration of web scraping
iteration+=1
print("START SCRAPING ON WHOLE DATASET:")
for i in missing:
    quotes, discussions, questions = get_discussion_stats(df['bookID'][i], row_number=i)

    if not pd.isnull(quotes):
        df.loc[i, 'quotes'] = quotes
    if not pd.isnull(discussions):
        df.loc[i, 'discussions'] = discussions
    if not pd.isnull(questions):
        df.loc[i, 'questions'] = questions

# Save backup of the iteration
#df.to_csv(f"{BASE_PATH}/discussion/books_discussion_final_{iteration}.csv")

missed  1
START SCRAPING ON WHOLE DATASET:
0 -- found on row : 6396  0.01% missing
None   None    None


### Re execute the previous cell, until you get all needed data

In [104]:
# Save latest version
df.to_csv(f"{BASE_PATH}/discussion/books_discussion_latest.csv")

### Merge vincent's scrape with mine

In order to accelerate the web scraping task, we share our results between two iterations from time to time.

In [61]:
df_vinz = pd.read_csv("discussion/books_discussion_final_6.csv")
df_vinz.isnull().sum()

Unnamed: 0.3             0
Unnamed: 0.2             0
Unnamed: 0.1             0
Unnamed: 0               0
bookID                   0
title                    0
authors                  0
average_rating           0
isbn                     0
isbn13                   0
language_code            0
num_pages                0
ratings_count            0
text_reviews_count       0
publication_date         0
publisher                0
quotes                2806
discussions           2806
questions             2806
dtype: int64

In [62]:
found_by_vinz = df_vinz[df_vinz['quotes'].isnull() == False]
missed_by_me= df[df['quotes'].isnull() == True].index
common_indexes = set(found_by_vinz.index).intersection(missed_by_me)
print(f"{len(found_by_vinz)} found by vincent")
print(f"{len(missed_by_me)} missed by me")
len(common_indexes)

8321 found by vincent
883 missed by me


145

In [63]:
for i in common_indexes:
    df.loc[i,'quotes'] = found_by_vinz.loc[i,'quotes']
    df.loc[i,'discussions'] = found_by_vinz.loc[i,'discussions']
    df.loc[i,'questions'] = found_by_vinz.loc[i,'questions']


In [64]:
df.isnull().sum()

Unnamed: 0.1            0
Unnamed: 0              0
bookID                  0
title                   0
authors                 0
average_rating          0
isbn                    0
isbn13                  0
language_code           0
num_pages               0
ratings_count           0
text_reviews_count      0
publication_date        0
publisher               0
quotes                738
discussions           738
questions             738
dtype: int64

### Part 2: Transform

In [105]:
# Load the latest csv
filename = f"{BASE_PATH}/discussion/books_discussion_latest.csv"
df_final = pd.read_csv(filename, sep=',', on_bad_lines="warn")
print(df_final.shape)
df_final.head()

(11127, 19)


,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,quotes,discussions,questions
0,0,0,0,0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,882,194,74
1,1,1,1,1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"1,152",307,107
2,2,2,2,2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,745,483,127
3,3,3,3,3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,747,417,121
4,4,4,4,4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,1,20,2


During scraping, some useless unnamed columns containing indexes were added, let's drop them

In [107]:
df_final.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3' ], inplace=True)

In [108]:
df_final.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11127.000000,11127.000000,1.112700e+04,11127.000000,1.112700e+04,11127.000000
mean,21310.938887,3.933631,9.759888e+12,336.376921,1.793641e+04,541.854498
std,13093.358023,0.352445,4.428964e+11,241.127305,1.124794e+05,2576.176608
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10287.000000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780586e+12,299.000000,7.450000e+02,46.000000
75%,32104.500000,4.135000,9.780873e+12,416.000000,4.993500e+03,237.500000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [109]:
# Added columns: quotes, discussions and questions do not figure in describe, because they are not numerical.
df_final.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
num_pages               int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
quotes                 object
discussions            object
questions              object
dtype: object

In [111]:
# we notice 'Add' value when there is no quote, replace it by '0'
df_final['quotes'].value_counts()

quotes
Add      4071
1         863
2         443
3         392
4         299
         ... 
747         1
425         1
672         1
301         1
1,641       1
Name: count, Length: 482, dtype: int64

In [113]:
# Same for discussions, 'Start' means 0 discussions
df_final['discussions'].value_counts()

discussions
Start    5198
1        1425
2         725
3         439
4         314
         ... 
128         1
870         1
314         1
1,048       1
1,047       1
Name: count, Length: 233, dtype: int64

In [114]:
# 'Ask' means 0 questions
df_final['questions'].value_counts()

questions
Ask    5706
1      1676
2       783
3       480
4       340
       ... 
40        1
63        1
61        1
60        1
245       1
Name: count, Length: 112, dtype: int64

### Replace Add, Start and Ask by 0, and at the same time remove ',' from other values (1,552 -> 1552)

In [122]:
df_final['quotes'] = df_final['quotes'].str.replace('Add', '0').str.replace(',', '')
df_final['quotes'].value_counts()

quotes
0       4071
1        863
2        443
3        392
4        299
        ... 
747        1
425        1
672        1
301        1
1641       1
Name: count, Length: 482, dtype: int64

In [123]:
df_final['discussions'] = df_final['discussions'].str.replace('Start', '0').str.replace(',', '')
df_final['discussions'].value_counts()

discussions
0       5198
1       1425
2        725
3        439
4        314
        ... 
128        1
870        1
314        1
1048       1
1047       1
Name: count, Length: 233, dtype: int64

In [124]:
df_final['questions'] = df_final['questions'].str.replace('Ask', '0').str.replace(',', '')
df_final['questions'].value_counts()

questions
0      5706
1      1676
2       783
3       480
4       340
       ... 
40        1
63        1
61        1
60        1
245       1
Name: count, Length: 112, dtype: int64

#### Convert the columns type from object to int

In [125]:
df_final['quotes'] = pd.to_numeric(df_final['quotes'], errors='coerce')
df_final['discussions'] = pd.to_numeric(df_final['discussions'], errors='coerce')
df_final['questions'] = pd.to_numeric(df_final['questions'], errors='coerce')
df_final.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
num_pages               int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
quotes                float64
discussions           float64
questions             float64
dtype: object

In [127]:
df_final.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count,quotes,discussions,questions
count,11127.000000,11127.000000,1.112700e+04,11127.000000,1.112700e+04,11127.000000,11126.000000,11126.000000,11126.000000
mean,21310.938887,3.933631,9.759888e+12,336.376921,1.793641e+04,541.854498,68.099676,18.728564,5.830937
std,13093.358023,0.352445,4.428964e+11,241.127305,1.124794e+05,2576.176608,225.334775,76.414218,20.968859
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,10287.000000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000,0.000000,0.000000,0.000000
50%,20287.000000,3.960000,9.780586e+12,299.000000,7.450000e+02,46.000000,3.000000,1.000000,0.000000
75%,32104.500000,4.135000,9.780873e+12,416.000000,4.993500e+03,237.500000,29.000000,5.000000,3.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000,2801.000000,1824.000000,553.000000


 ### Save df_final

In [128]:
filename = f"{BASE_PATH}/discussion/books_discussion_latest_cleaned.csv"
df_final.to_csv(filename)